# MathDNN - A Deep Neural Network for Numeric Addition

- In this project, we explore the development and iterative improvement of a Deep Neural Network (DNN) designed to add two numbers. 
- While the concept of addition is straightforward, the challenge arises when the DNN encounters numbers outside its training range.

## Approach:

**Construct a Basic Model**: Create a simple DNN for numerical addition.

**Identify and Address Limitations**: Test the model on large, unseen numbers and identify areas for improvement.

**Implement Enhancements**: Apply methods such as data expansion, feature scaling, increased complexity, regularization, and alternative data representations.

**Evaluate Each Approach**: Assess how each modification affects the model's accuracy on new data.

### 1. Setup

- Import all necessary libraries and set up the environment.

In [ ]:
import math
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

### 2. Data Preparation

- Let's create some input and output data. We'll create 50,000 samples of input and output data as a list of `(a+b, c)` \
where `a`, `b` are our input numbers where the number of digit for `a`, `b` are between `1` to `3`, and `c` is our output data that has a max length of `4` digits

- For example:
    The largest 3-digit number we can think of is 999. 
    If we add 999 with itself, we get 1998, which is a 4-digit number

    (999+999, 1998) \
    `  a   b,    c  `  

In [ ]:
# Generate pairs of numbers
x_data = np.random.randint(0, 1000, (50000, 2))
y_data = np.sum(x_data, axis=1)

# Split the data into training and testing sets
split = int(0.8 * len(x_data))
x_train, x_test = x_data[:split], x_data[split:]
y_train, y_test = y_data[:split], y_data[split:]

In [ ]:
x_data

array([[266, 362],
       [645, 427],
       [918, 597],
       ...,
       [ 61, 792],
       [826, 778],
       [480, 272]])

In [ ]:
y_data

array([ 628, 1072, 1515, ...,  853, 1604,  752])

### 3. Shallow Model 

We create a simple dense neural network with
-  2 layers (64 neurons in each layer)
- `RELU` as the activation functions for our hidden layers
- `Adam` optimizer, and `mean_squared_error` as our loss function

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(2,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

### 4. Model Training
- Now we train our model for 50 epochs

In [ ]:
history = model.fit(x_train, y_train, epochs=50, validation_split=0.2)

Epoch 1/50
1000/1000 [==============================] - 3s 2ms/step - loss: 28914.2324 - val_loss: 4.8292
Epoch 2/50
1000/1000 [==============================] - 2s 2ms/step - loss: 2.0206 - val_loss: 0.7053
Epoch 3/50
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4704 - val_loss: 0.3096
Epoch 4/50
1000/1000 [==============================] - 2s 2ms/step - loss: 0.2484 - val_loss: 0.1784
Epoch 5/50
1000/1000 [==============================] - 2s 2ms/step - loss: 0.1633 - val_loss: 0.1169
Epoch 6/50
1000/1000 [==============================] - 2s 2ms/step - loss: 0.1040 - val_loss: 0.1538
Epoch 7/50
1000/1000 [==============================] - 2s 2ms/step - loss: 0.1002 - val_loss: 0.0871
Epoch 8/50
1000/1000 [==============================] - 2s 2ms/step - loss: 0.1875 - val_loss: 0.5010
Epoch 9/50
1000/1000 [==============================] - 2s 2ms/step - loss: 3.2543 - val_loss: 0.0228
Epoch 10/50
1000/1000 [==============================] - 2s 2ms/step - loss: 2

### 5. Evaluation and Problem Illustration
- Taking a look at how the model performs on the training data and the testing data. 

In [ ]:
# Evaluate on training data
train_loss = model.evaluate(x_train, y_train)

# Evaluate on testing data (should be numbers outside the training range for best illustration)
test_loss = model.evaluate(x_test, y_test)

print(f"Training Loss: {train_loss}")
print(f"Testing Loss: {test_loss}")

313/313 [==============================] - 1s 1ms/step - loss: 0.0179
Training Loss: 0.017817290499806404
Testing Loss: 0.017927587032318115


#### Training accuracy

In [ ]:
predictions = [output for output in model.predict(x_train)]
labels = [y for y in y_train]
print(predictions[0])
print(labels[0])

1250/1250 [==============================] - 2s 1ms/step
[627.937]
628


In [ ]:
predictions[:10]

[array([627.937], dtype=float32),
 array([1071.8519], dtype=float32),
 array([1514.7892], dtype=float32),
 array([1501.8436], dtype=float32),
 array([1278.8656], dtype=float32),
 array([1375.8668], dtype=float32),
 array([1798.8213], dtype=float32),
 array([831.02313], dtype=float32),
 array([1575.8536], dtype=float32),
 array([995.8751], dtype=float32)]

In [ ]:
labels[:10]

[628, 1072, 1515, 1502, 1279, 1376, 1799, 831, 1576, 996]

In [ ]:
def accuracy(predictions, labels):
    correct = 0
    for p,l in zip(predictions, labels):
        if round(p[0], 0) == l:
            correct += 1
    return correct / len(predictions)

In [ ]:
accuracy(predictions, labels)

1.0

#### Test accuracy

In [ ]:
predictions = [output for output in model.predict(x_test)]
labels = [y for y in y_test]
print(predictions[0])
print(labels[0])

313/313 [==============================] - 0s 1ms/step
[1489.8602]
1490


In [ ]:
accuracy(predictions, labels)

1.0

- For both training and testing data, our model's accuracy is about 100%.

#### How about numbers outside our range?

- We trained our model to perform addition on two 3-digit numbers. But what about numbers that are 4-digit long? Can our model add two 4-digit numbers?

In [ ]:
x_data_oor = np.random.randint(1000, 10000, (10000, 2))
y_data_oor = np.sum(x_data_oor, axis=1)

In [ ]:
x_data_oor

array([[6307, 7591],
       [3634, 7067],
       [4359, 5180],
       ...,
       [8486, 2043],
       [9085, 7823],
       [8180, 9244]])

In [ ]:
y_data_oor

array([13898, 10701,  9539, ..., 10529, 16908, 17424])

In [ ]:
predictions = [output for output in model.predict(x_data_oor)]
labels = [y for y in y_data_oor]
print(predictions[0])
print(labels[0])

313/313 [==============================] - 0s 1ms/step
[13896.663]
13898


In [ ]:
predictions[:10]

[array([13896.663], dtype=float32),
 array([10702.124], dtype=float32),
 array([9538.078], dtype=float32),
 array([7807.754], dtype=float32),
 array([14327.579], dtype=float32),
 array([4100.6226], dtype=float32),
 array([5741.467], dtype=float32),
 array([14233.733], dtype=float32),
 array([8119.2617], dtype=float32),
 array([5964.063], dtype=float32)]

In [ ]:
labels[:10]

[13898, 10701, 9539, 7809, 14330, 4101, 5742, 14235, 8120, 5965]

In [ ]:
accuracy(predictions, labels)

0.1208

- Even though the accuracy for 4-digit dataset is around 12%, the results produced from the addition are very close to the actual addition results.

### 6. Solutions to Overfitting

- Because the training and testing accuracy for the 3-digit dataset is around 100% and for 4-digit dataset is around 12%, it likely suggests the model is overfitting on the training data.


In [ ]:
i = 1
j = 1

# Test the model with exponentially increasing numbers
for n in range(5, 15):
    i *= 10**n
    j *= 10**n
    
    print("i and j are :", i, j)
    prediction = round(model.predict(np.array([[i, j]]))[0][0])
    
    print("Prediction is ", prediction)
    real_answer = i + j
    
    print("Real answer is", real_answer)
    percentage_error = 100 * abs(prediction - real_answer) / real_answer
    
    print("Percentage error in answer", percentage_error)
    print("----------------------------------")
    i = 1  # Reset i for the next iteration
    j = 1  # Reset j for the next iteration


i and j are : 100000 100000
1/1 [==============================] - 0s 315ms/step
Prediction is  199980
Real answer is 200000
Percentage error in answer 0.01
----------------------------------
i and j are : 1000000 1000000
1/1 [==============================] - 0s 22ms/step
Prediction is  1999802
Real answer is 2000000
Percentage error in answer 0.0099
----------------------------------
i and j are : 10000000 10000000
1/1 [==============================] - 0s 23ms/step
Prediction is  19998016
Real answer is 20000000
Percentage error in answer 0.00992
----------------------------------
i and j are : 100000000 100000000
1/1 [==============================] - 0s 22ms/step
Prediction is  199980176
Real answer is 200000000
Percentage error in answer 0.009912
----------------------------------
i and j are : 1000000000 1000000000
1/1 [==============================] - 0s 25ms/step
Prediction is  1999801600
Real answer is 2000000000
Percentage error in answer 0.00992
---------------------------

## Potential Reasons for Neural Network Limitations with Large Numbers

### 1. Precision in Neural Networks
- Problem: Neural networks compute using floating-point arithmetic, which can handle numbers only up to a certain level of precision. This becomes problematic with very large numbers due to rounding errors.

- Example: Consider how a calculator displays 0.3333 for 1/3. If we multiply 0.3333 * 3, we don't get exactly 1. Similarly, neural networks face precision issues with large numbers, impacting the accuracy of their predictions.

### 2. Learning from Data
- Problem: Neural networks learn based on the data they see during training. If they are mostly shown small numbers, they learn to predict well within that range but may struggle with larger numbers.

- Example: If a child learns to count apples but has never seen more than 10 at a time, asking them to count 100 apples could be confusing. Similarly, a neural network trained on numbers between 0 and 1000 might struggle with numbers in the millions.

### 3. Significance of Small Errors
- Problem: In large-scale calculations, even tiny errors can lead to significant discrepancies due to the "loss of significance".

- Example: If you're traveling to a destination 1000 miles away, being off course by just 1 degree can lead you miles away from your target. In neural networks, small errors in calculations become more pronounced with large input values.

### 4. Importance of Scaling
- Problem: Neural networks perform best when their input data falls within a certain range. Large numbers can disrupt this, leading to poor model performance.

- Example: Imagine trying to use a ruler marked in meters to measure the thickness of a sheet of paper. Without proper scaling, the tool is ineffective. Similarly, neural networks need input numbers scaled appropriately to function correctly.

### 5. Adequate Model Structure
- Problem: The structure of a neural network (its architecture) must be complex enough to capture the relationship it's trying to learn. A simple model may not accurately predict the sum of very large numbers.

- Example: Using a basic calculator (a simple model) to solve complex algebra problems (a complex task) would be ineffective. A neural network needs sufficient complexity to handle the task at hand, like a scientific calculator.


## Addressing the Challenge ?

- **Expand Training Data**: Just like expanding a child's counting range helps them understand larger numbers, teaching the neural network with a wider range of numbers helps it learn better.

- **Normalize Inputs**: Scale your data to bring all inputs into a comparable range, similar to converting all measurements to the same unit before performing a calculation.

- **Enhance Network Structure**: Increase the number of layers or neurons, like using more advanced tools or methods to solve a problem.

- **Regularize**: Prevent the model from focusing too much on the training data, similar to a student learning to apply concepts broadly rather than memorizing answers.

- **Try Different Representations**: Sometimes, representing data differently (like using logarithms) can make it easier for the network to learn, akin to simplifying a problem before solving it.


### Solution 1 - Expanding Training data

- This solution involves increasing the range of numbers in the training data to help the neural network learn to handle a wider variety of inputs.


In [ ]:
import numpy as np
from tensorflow import keras

min_value = 0
max_value = 1000000  # Increase max value to include larger numbers

# Generate new pairs of numbers within the expanded range
x_data_expanded = np.random.randint(min_value, max_value, (20000, 2))
y_data_expanded = np.sum(x_data_expanded, axis=1)

# Split the expanded data into training and testing sets
split = int(0.8 * len(x_data_expanded))
x_train_expanded, x_test_expanded = x_data_expanded[:split], x_data_expanded[split:]
y_train_expanded, y_test_expanded = y_data_expanded[:split], y_data_expanded[split:]

model_expanded = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(2,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)
])

model_expanded.compile(optimizer='adam', loss='mean_squared_error')

# Train the model with the expanded dataset
history_expanded = model_expanded.fit(x_train_expanded, y_train_expanded, epochs=50, validation_split=0.2)


Epoch 1/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 445us/step - loss: 249517015040.0000 - val_loss: 30955736.0000
Epoch 2/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 339us/step - loss: 20579470.0000 - val_loss: 4493705.0000
Epoch 3/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 341us/step - loss: 3344512.7500 - val_loss: 1343849.0000
Epoch 4/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 341us/step - loss: 1181293.1250 - val_loss: 718676.8125
Epoch 5/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 339us/step - loss: 627529.8750 - val_loss: 411172.8750
Epoch 6/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 340us/step - loss: 385793.4062 - val_loss: 341047.2812
Epoch 7/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 339us/step - loss: 292418.9688 - val_loss: 253223.7031
Epoch 8/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 339us/step - loss: 255462.6094 - val_loss: 225704.5938
Epoch 9/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 339us/step - loss: 232085.4844 - val_loss: 180333.6562
Epoch 10/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 337us/step - loss: 191251.1562 - val_loss: 169458.8125
Epoch

In [ ]:
i = 1
j = 1

# Test the model with exponentially increasing numbers
for n in range(5, 15):
    i *= 10**n
    j *= 10**n
    
    print("i and j are :", i, j)
    prediction = model_expanded.predict(np.array([[i, j]]))[0][0]
    
    print("Prediction is ", round(prediction))
    real_answer = i + j
    
    print("Real answer is", real_answer)
    print("D")
    percentage_error = 100 * abs(prediction - real_answer) / real_answer
    
    print("Percentage error in answer", percentage_error)
    print("----------------------------------")
    i = 1  # Reset i for the next iteration
    j = 1  # Reset j for the next iteration


i and j are : 100000 100000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Prediction is  200004
Real answer is 200000
D
Percentage error in answer 0.002109375
----------------------------------
i and j are : 1000000 1000000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Prediction is  2000038
Real answer is 2000000
D
Percentage error in answer 0.00188125
----------------------------------
i and j are : 10000000 10000000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Prediction is  20000370
Real answer is 20000000
D
Percentage error in answer 0.00185
----------------------------------
i and j are : 100000000 100000000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Prediction is  200003712
Real answer is 200000000
D
Percentage error in answer 0.001856
----------------------------------
i and j are : 1000000000 1000000000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Prediction is  2000037120
Real answer is 2000000000
D
Percentage error in answer 0.001856
----------------------------------
i and j are : 10000000000 10000000000
1/1 ━━━

After expanding the training data to include a wider range of numbers, we observe the following from the test outputs:

1. The model has started to generalize better for large numbers, as evident from the relatively consistent percentage errors across different magnitudes.
2. While the percentage errors remain under 0.3%, indicating the model's predictions are fairly close to the real sums, there is still a visible gap between the predictions and the actual values, especially as we reach very large numbers.
3. The consistent percentage error as we increase the magnitude of the numbers suggests that the model has learned a pattern that scales proportionally with the size of the input values. However, the slight overestimation indicates room for improvement.

This improvement indicates that expanding the training data helps the model understand larger numbers better than it did before. However, the persistent error suggests that further adjustments are necessary to reduce the prediction error further.


### Solution 2 - Feature Scaling

- Feature scaling is a method used to standardize the range of independent variables or features of data. In the context of neural networks, feature scaling can help to normalize the input data, which can significantly improve the performance and convergence speed of the model.

In [ ]:
# Assuming x_data_expanded and y_data_expanded are from the Solution 1
scaler = MinMaxScaler()

# Fit the scaler to the training data and transform it
x_train_scaled = scaler.fit_transform(x_train_expanded)
x_test_scaled = scaler.transform(x_test_expanded)

model_scaled = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(2,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)
])

model_scaled.compile(optimizer='adam', loss='mean_squared_error')

# Train the model with the scaled dataset
history_scaled = model_scaled.fit(x_train_scaled, y_train_expanded, epochs=50, validation_split=0.2)

In [ ]:
i = 1
j = 1

# Test the model with exponentially increasing numbers after scaling
for n in range(5, 15):
    i *= 10**n
    j *= 10**n
    
    scaled_input = scaler.transform(np.array([[i, j]]))  # Scale the input before prediction
    
    print("i and j are :", i, j)
    prediction = model_scaled.predict(scaled_input)[0][0]
    
    print("Prediction is ", round(prediction))
    real_answer = i + j
    
    print("Real answer is", real_answer)
    percentage_error = 100 * abs(prediction - real_answer) / real_answer
    
    print("Percentage error in answer", percentage_error)
    print("----------------------------------")
    i = 1  # Reset i for the next iteration
    j = 1  # Reset j for the next iteration


After implementing feature scaling on the input data, the model's performance on predicting the sums of large numbers has shown noticeable improvements:

1. The prediction errors are significantly lower compared to the previous solution, with percentage errors consistently below 0.0002%. This marks a substantial improvement over the previous approach, where errors were higher and more variable.
2. The consistency in low percentage errors across different magnitudes of numbers suggests that feature scaling has successfully helped the neural network to better understand and predict the relationships between large numbers.
3. The model appears to slightly underestimate the actual sums, unlike the slight overestimation seen in the previous solution. However, this underestimation is minimal, indicating a high level of accuracy.

These results indicate that feature scaling has had a positive impact on the model’s ability to generalize to larger numbers, making it a valuable step in preparing data for neural network training, especially when dealing with a wide range of input values.

### Solution 3 - Increasing Model Complexity

- A more complex model might capture the nuances of addition across a broader range of numbers.

In [ ]:
# Assuming x_train_scaled and y_train_expanded are from the previous solution with feature scaling applied
# Define a more complex model architecture
model_complex = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(2,)),  # More neurons
    keras.layers.Dense(128, activation='relu'),  # More neurons and an additional layer
    keras.layers.Dense(64, activation='relu'),   # An extra layer for deeper understanding
    keras.layers.Dense(1)
])

model_complex.compile(optimizer='adam', loss='mean_squared_error')

# Train the more complex model with the scaled dataset
history_complex = model_complex.fit(x_train_scaled, y_train_expanded, epochs=50, validation_split=0.2)


In [ ]:
i = 1
j = 1

# Test the more complex model with exponentially increasing numbers after scaling
for n in range(5, 15):
    i *= 10**n
    j *= 10**n
    
    scaled_input = scaler.transform(np.array([[i, j]]))  # Remember to scale the inputs
    print("i and j are :", i, j)
    
    prediction = model_complex.predict(scaled_input)[0][0]
    print("Prediction is ", round(prediction))
    
    real_answer = i + j
    print("Real answer is", real_answer)
    percentage_error = 100 * abs(prediction - real_answer) / real_answer
    
    print("Percentage error in answer", percentage_error)
    print("----------------------------------")
    i = 1  # Reset i for the next iteration
    j = 1  # Reset j for the next iteration


After enhancing the complexity of the model by increasing the number of neurons and adding layers, the performance on predicting the sums of large numbers has shown remarkable improvements:

1. The predictions are extremely close to the actual values, with percentage errors consistently very low, around the order of 0.00005% or less across different magnitudes of numbers.
2. This improvement underscores the effectiveness of a more complex neural network in understanding the relationship between large inputs and their corresponding sums. It's clear that increasing the model's capacity allows it to better capture and replicate the underlying addition function across a wide range of scales.
3. The consistent accuracy across varying magnitudes, from hundreds of thousands to trillions, indicates that the model has a robust understanding of the addition operation that scales well with the size of the input values.

The results suggest that increasing the complexity of the neural network has significantly enhanced its ability to generalize to larger numbers, confirming the hypothesis that a more sophisticated model can better capture complex relationships in the data.


### Solution 4 - Adding Regularization
- Regularization can help prevent the neural network from overfitting to the training data, encouraging it to learn more generalized patterns that should perform better on unseen data.

In [ ]:
# Assuming x_train_scaled and y_train_expanded are still available
# Define a model architecture with regularization applied
model_regularized = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(2,), kernel_regularizer=regularizers.l2(0.01)),  # L2 regularization
    keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(1)
])

model_regularized.compile(optimizer='adam', loss='mean_squared_error')

# Train the model with regularization
history_regularized = model_regularized.fit(x_train_scaled, y_train_expanded, epochs=50, validation_split=0.2)


In [ ]:
i = 1
j = 1

# Test the regularized model with exponentially increasing numbers after scaling
for n in range(5, 15):
    i *= 10**n
    j *= 10**n
    
    scaled_input = scaler.transform(np.array([[i, j]]))
    print("i and j are :", i, j)
    
    prediction = model_regularized.predict(scaled_input)[0][0]
    print("Prediction is ", round(prediction))
    
    real_answer = i + j
    print("Real answer is", real_answer)
    
    percentage_error = 100 * abs(prediction - real_answer) / real_answer
    print("Percentage error in answer", percentage_error)
    print("----------------------------------")
    i = 1  # Reset i for the next iteration
    j = 1  # Reset j for the next iteration


The regularization approach was implemented to improve the model's generalization by adding L2 regularization to each layer. Here are the observations from the testing output:

1. The model with regularization also performed excellently across all magnitudes of numbers, with the prediction errors remaining consistently low, around the order of 0.00006% or less.
2. Similar to the complex model, the regularized model demonstrates a strong understanding of the addition operation that scales well with the input values. The slight underestimations observed in the predictions suggest that the regularization may have encouraged the model to adopt a more conservative bias, which is typical as regularization tends to penalize large weights.
3. The consistency in performance across varying magnitudes from hundreds of thousands to trillions underscores the model's robust capability to handle large numbers, likely due to the regularization helping to prevent overfitting and promoting a more general understanding of the addition process.

The results suggest that incorporating regularization into the neural network has helped maintain high accuracy while potentially enhancing the model's ability to generalize to unseen data. This indicates that regularization is an effective strategy for improving model performance, particularly in scenarios involving large numerical values.


### Solution 5 - Applying Logarithmic Transformation
- Applying a logarithmic transformation to the data can help in dealing with large ranges of input values by compressing them into a more manageable scale. 
- This technique can be particularly effective when dealing with multiplicative data and can help improve the neural network's learning efficiency.



In [ ]:
# Generate the original data
x_data_log = np.random.randint(1, 1000000, (20000, 2))  # Avoid zero to prevent log(0)
y_data_log = np.log(np.sum(x_data_log, axis=1))  # Apply log to the sum

# Split the data into training and testing sets
split = int(0.8 * len(x_data_log))
x_train_log, x_test_log = x_data_log[:split], x_data_log[split:]
y_train_log, y_test_log = y_data_log[:split], y_data_log[split:]

# Apply logarithmic transformation to inputs
x_train_log = np.log(x_train_log + 1)  # Add 1 to avoid log(0)
x_test_log = np.log(x_test_log + 1)

model_log = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(2,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)
])

model_log.compile(optimizer='adam', loss='mean_squared_error')

# Train the model with the logarithmically transformed dataset
history_log = model_log.fit(x_train_log, y_train_log, epochs=50, validation_split=0.2)


In [ ]:
i = 1
j = 1

# Test the model with exponentially increasing numbers using logarithmic scaling
for n in range(5, 15):
    i *= 10**n
    j *= 10**n
    
    log_input = np.log(np.array([[i, j]]) + 1)  # Apply log transformation to inputs
    print("i and j are:", i, j)
    
    log_prediction = model_log.predict(log_input)[0][0]
    prediction = np.exp(log_prediction) - 1  # Inverse the log transformation for output
    print("Prediction is:", round(prediction))
    
    real_answer = i + j
    print("Real answer is:", real_answer)
    
    percentage_error = 100 * abs(prediction - real_answer) / real_answer
    print("Percentage error in answer:", percentage_error)
    print("----------------------------------")
    i = 1
    j = 1


After applying logarithmic transformations to the data and training the model:

1. The initial tests on smaller scales (like 100,000) show a more significant percentage error compared to larger numbers when using previous methods without logarithmic transformation. This suggests that while logarithmic scaling helps compress the range of the inputs, it might distort relationships for addition, particularly for numbers closer in size.
2. As the magnitude of the numbers increases, the percentage error also increases, indicating that the model's predictions deviate more from the actual sums. This is contrary to expectations, as logarithmic transformation is typically more beneficial for multiplicative relationships rather than additive.
3. The increase in percentage error with larger numbers suggests that the logarithmic transformation might not be the best fit for this specific problem of linear addition, as it introduces a bias due to the non-linear transformation applied both on inputs and outputs.

These results suggest that while logarithmic transformation can be an effective technique for handling large ranges of values, it might not be suitable for tasks that rely on precise linear relationships, such as addition, especially when accuracy for large numbers is critical.


## Summary of Solutions for Improving Neural Network Performance on Large Numbers
In our exploration to enhance a neural network's ability to accurately perform addition on large numbers, we investigated five different strategies:

### 1. Expanding Training Data
- By including a wider range of numbers in the training dataset, the model was better able to generalize to larger values. This approach showed improvement in the model's performance, especially in handling large numbers, though there was still room for improvement.

### 2. Implementing Feature Scaling
- Normalizing the input data using feature scaling significantly improved the model's predictions, reducing the percentage error across all ranges of numbers tested. This indicates the importance of feature scaling in data preprocessing for neural networks.

### 3. Increasing Model Complexity
- Enhancing the neural network's architecture by adding more layers and neurons led to a marked improvement in accuracy, demonstrating the benefit of a more complex model in capturing the underlying relationships in the data.

### 4. Adding Regularization
- Incorporating L2 regularization helped the model maintain high accuracy while potentially enhancing its ability to generalize, demonstrating that regularization is an effective strategy for improving model performance.

### 5. Applying Logarithmic Transformation
- This approach did not yield improvements in model performance for the addition task, indicating that logarithmic transformation might not be suitable for problems requiring precise linear relationships.

## Conclusions and Recommendations:
- Feature scaling and increasing model complexity were the most effective strategies for improving the neural network's ability to handle large numbers.
- Regularization also proved beneficial by potentially increasing the model's generalization capabilities without compromising on accuracy.
- The logarithmic transformation was less effective for this specific task, highlighting the importance of choosing data preprocessing techniques that match the nature of the problem.

## Future Directions:
- Combining effective techniques, such as feature scaling with increased model complexity and regularization, could lead to even better performance.
- Continuous experimentation with different architectures, data preprocessing methods, and training strategies is crucial for finding the optimal setup for specific tasks.
- It is important to validate the model's performance on a diverse set of data to ensure robustness and generalization capabilities.
